In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import glob
import datetime
import time
import numba
import scipy as sc

import bokeh
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter, ColumnDataSource, ColorBar
from bokeh.plotting import figure
from bokeh.layouts import gridplot, row, column
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()
import holoviews as hv
import bebi103
hv.extension('bokeh')

Windrose data source below 

I am using the NOAA data that has already been filtered for the fresh emission percentile 

All of this will be for Lamont area

## Doing example using NOAA data from propane bayes project 
See "All years" section for analysis using newer NOAA data (which has less data in comparison)

In [ ]:
# NOAA_flask = pd.read_csv('/Users/arianatribby/git/propane_bayes_local/data/processing/NOAAfreshemissions.csv')
# this is the wrong noaa data, see below for the right one 
NOAA_flask = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/processing/newNOAA_freshemiss.csv')



In [ ]:
NOAA_flask.head()

In [ ]:
noaa = NOAA_flask.loc[NOAA_flask['sample_site_code'] == 'SGP']

In [ ]:
noaa.tail()

First separate out the tower from the aircraft. See when each type of data was taken. 

In [ ]:
fh = 400
fw = 400

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='')

p.circle(noaa.completetime.values, noaa.sample_altitude.values*1e-3, 
         size=5, fill_alpha=.5)

p.xaxis[0].formatter = DatetimeTickFormatter(months=['%b %Y'],years=['%Y'])
# p.xaxis[0].formatter = DatetimeTickFormatter(days=['%b %Y'])
p.xaxis.axis_label = "flask sample time"
p.yaxis.axis_label = "sample altitude (km)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
bokeh.io.show(p)

# formatting https://docs.bokeh.org/en/2.4.2/docs/reference/models/formatters.html 

In [ ]:
noaa_tower = noaa.loc[noaa['sample_altitude'] < 400]

Also, tccon starts in 2017 summer, so start with that. Going to only analyze the summer data. But you don't have summer data for 2018 at the moment. Waiting for NOAA to send data. 

In [ ]:
noaatowersummer = noaa_tower.loc[(noaa_tower['dt_time'] > '2017-06') & 
                                 (noaa_tower['dt_time'] < '2017-10')]

What time of the day did these measurements occur

In [ ]:
noaatowersummer.dt_time.values

In [ ]:
noaatowersummer.tail()

I double checked the raw file. Before 2017, there were more flasks that were taken at different times (even after filtering for tower observations), but beyond 2017 there were only 18, 19, and 20 th hour flask. That is not taking into account the filtering for fresh emissions. So I believe this (I thought maybe there was an issue with the time values, but the raw file had the time hour written out and this variable was just a compilation of the time year, month, day, hour). 

So tccon has best observations at noon, but this doesn't have that. oh well? 

### Wind data

Getting correlations between NOAA tower and aircraft flask with wind roses from https://mesonet.agron.iastate.edu/sites/windrose.phtml?station=WDG&network=OK_ASOS and you clicked on custom wind rose https://mesonet.agron.iastate.edu/request/download.phtml?network=OK_ASOS

And selecting the blackwell station because that seems to be the closest. Selected the variables of interest and utc time. 


In [ ]:
wind = pd.read_csv('../../data/windrose/BKN.csv')

In [ ]:
wind.tail()

In [ ]:
newwindtime = [datetime.datetime.timestamp(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M")) for x in wind.valid.values]

In [ ]:
wind['timestamp'] = newwindtime

In [ ]:
f = sc.interpolate.interp1d(wind.timestamp.values,wind.drct.values, kind='nearest')
g = sc.interpolate.interp1d(wind.timestamp.values,wind.sped.values, kind='nearest')

In [ ]:
newdirct = f(noaatowersummer.completetime.values)
newspd = g(noaatowersummer.completetime.values)

In [ ]:
newspd

In [ ]:
newdirct

The reason these are nan is because the data is nan for the wind direction near those time values.  

In [ ]:
noaatowersummer.dt_time.values[9:12]

In [ ]:
noaatowersummer.dt_time.values[19:22]

In [ ]:
newdirct[10] = 200.
newdirct[20] = 180.

color this by wind direction


In [ ]:
fh = 400
fw = 400

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='June-Sept 2017 Tower SGP')

source = ColumnDataSource(dict(x=noaatowersummer.c2h6.values*1e-3, 
                               y=noaatowersummer.c3h8.values*1e-3, 
                               z=newdirct))
mapper = linear_cmap(field_name='z', palette=Viridis[5],
                     low=min(newdirct) ,high=max(newdirct))
p.circle(x='x',y='y',line_color=mapper,color=mapper,fill_alpha=.5,
         size=10,source=source)
color_bar = ColorBar(color_mapper=mapper['transform'],
                     width=8, title='Wind direction (degrees)',
                     major_label_text_font_size="15pt",
                     title_standoff=20,
                     label_standoff=5,
                     title_text_font_size="15pt",
                     location=(0,0))
p.add_layout(color_bar, 'right')
p.xaxis.axis_label = "C₂H₆ (ppb)"
p.yaxis.axis_label = "C₃H₈ (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.title.text_font_size = "16pt"
bokeh.io.show(p)


size of glyphs - relative magnitude of wind speed

In [ ]:
fh = 400
fw = 400

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='June-Sept 2017 Tower SGP')

source = ColumnDataSource(dict(x=noaatowersummer.c2h6.values*1e-3, 
                               y=noaatowersummer.c3h8.values*1e-3, 
                               z=newdirct,
                               q=newspd/20))
mapper = linear_cmap(field_name='z', palette=Viridis[5],
                     low=min(newdirct) ,high=max(newdirct))
p.circle(x='x',y='y',line_color=mapper,color=mapper,fill_alpha=.5,
         radius='q',source=source)
color_bar = ColorBar(color_mapper=mapper['transform'],
                     width=8, title='Wind direction (degrees)',
                     major_label_text_font_size="15pt",
                     title_standoff=20,
                     label_standoff=5,
                     title_text_font_size="15pt",
                     location=(0,0))
p.add_layout(color_bar, 'right')
p.xaxis.axis_label = "C₂H₆ (ppb)"
p.yaxis.axis_label = "C₃H₈ (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.title.text_font_size = "16pt"
bokeh.io.show(p)



In [ ]:
len(noaatowersummer)

### Now repeat for aircraft

In [ ]:
noaa_plane = noaa.loc[noaa['sample_altitude'] > 400]

Plane had limited observations. Check when they occured, only look at the summer data. 

Not good, you don't have anything for 2017-2018 

#### Prepare data for footprints 
Prepare a csv file that has 4 columns: lat,lon,time,height - all this is needed to make the footprints.

In [ ]:
noaatowersummer.columns

STILT needs sample_intake_height

In [ ]:
towertocsv = noaatowersummer[['lat','lon','dt_time','completetime','sample_intake_height']]

In [ ]:
towertocsv.to_csv('../../data/windrose/noaa_summer2017tower_varsforfootprint.csv')

# All years 

In [ ]:
wind2017 = wind.copy()

In [ ]:
wind2018 = pd.read_csv('../../data/windrose/BKN-2018.csv')
wind2019 = pd.read_csv('../../data/windrose/BKN-2019.csv')
wind2020 = pd.read_csv('../../data/windrose/BKN-2020.csv')
wind2021 = pd.read_csv('../../data/windrose/BKN-2021.csv')

In [ ]:
wind2017.drop(['peak_wind_drct','peak_wind_time'],axis=1,inplace=True)

Put everything in a single dataframe

In [ ]:
# 2021 data is good, but not including it in since only have NOAA until 2020

wind_all_years = pd.concat([wind2017, wind2018, wind2019, wind2020], ignore_index=True)

Generate timestamp for interpolation to noaa time

In [ ]:
newwindtime = [datetime.datetime.timestamp(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M")) for x in wind_all_years.valid.values]

In [ ]:
wind_all_years['timestamp'] = newwindtime

In [ ]:
f = sc.interpolate.interp1d(wind_all_years.timestamp.values,wind_all_years.drct.values, kind='nearest')
g = sc.interpolate.interp1d(wind_all_years.timestamp.values,wind_all_years.sped.values, kind='nearest')

Grab noaa fresh data for all years 

In [ ]:
NOAA_fresh_summerallyears = pd.read_csv('../../data/processing/newNOAA_freshemiss2016to2020.csv')
NOAAoldandfresh_allyears = pd.read_csv('../../data/processing/newNOAA_oldandfreshemiss2016to2020.csv')

Remove 2016 NOAA data since won't have tccon data for that time period

In [ ]:
NOAA_fresh_summerallyears.drop(NOAA_fresh_summerallyears.loc[NOAA_fresh_summerallyears['year'] == 2016].index, inplace=True)
NOAAoldandfresh_allyears.drop(NOAAoldandfresh_allyears.loc[NOAAoldandfresh_allyears['year'] == 2016].index, inplace=True)

In [ ]:
newdirct = f(NOAA_fresh_summerallyears.completetime.values)
newspd = g(NOAA_fresh_summerallyears.completetime.values)

newdirct_all = f(NOAAoldandfresh_allyears.completetime.values)
newspd_all = g(NOAAoldandfresh_allyears.completetime.values)

Check for NaNs (not due to NOAA but due to missing met data)

In [ ]:
newdirct_all

In [ ]:
newspd

Linearly interpolate data

In [ ]:
mask = np.isnan(newdirct)
newdirct[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), newdirct[~mask])

print(newdirct)

In [ ]:
mask = np.isnan(newdirct_all)
newdirct_all[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), newdirct_all[~mask])

print(newdirct_all)

color this by wind direction


In [ ]:
fh = 400
fw = 400

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='2017-2020 Tower+Plane SGP')

source = ColumnDataSource(dict(x=NOAA_fresh_summerallyears.c2h6.values*1e-3, 
                               y=NOAA_fresh_summerallyears.c3h8.values*1e-3, 
                               z=newdirct))
mapper = linear_cmap(field_name='z', palette=Viridis[5],
                     low=min(newdirct) ,high=max(newdirct))
p.circle(x='x',y='y',line_color=mapper,color=mapper,fill_alpha=.5,
         size=10,source=source)
color_bar = ColorBar(color_mapper=mapper['transform'],
                     width=8, title='Wind direction (degrees)',
                     major_label_text_font_size="15pt",
                     title_standoff=20,
                     label_standoff=5,
                     title_text_font_size="15pt",
                     location=(0,0))
p.add_layout(color_bar, 'right')
p.xaxis.axis_label = "C₂H₆ (ppb)"
p.yaxis.axis_label = "C₃H₈ (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.title.text_font_size = "16pt"
bokeh.io.show(p)


size of glyphs - relative magnitude of wind speed

In [ ]:
fh = 400
fw = 400

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='2017-2020 Tower+Plane SGP')

source = ColumnDataSource(dict(x=NOAA_fresh_summerallyears.c2h6.values*1e-3, 
                               y=NOAA_fresh_summerallyears.c3h8.values*1e-3, 
                               z=newdirct,
                               q=newspd/20))
mapper = linear_cmap(field_name='z', palette=Viridis[5],
                     low=min(newdirct) ,high=max(newdirct))
p.circle(x='x',y='y',line_color=mapper,color=mapper,fill_alpha=.5,
         radius='q',source=source)
color_bar = ColorBar(color_mapper=mapper['transform'],
                     width=8, title='Wind direction (degrees)',
                     major_label_text_font_size="15pt",
                     title_standoff=20,
                     label_standoff=5,
                     title_text_font_size="15pt",
                     location=(0,0))
p.add_layout(color_bar, 'right')
p.xaxis.axis_label = "C₂H₆ (ppb)"
p.yaxis.axis_label = "C₃H₈ (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.title.text_font_size = "16pt"
bokeh.io.show(p)



In [ ]:
NOAA_fresh_summerallyears.groupby(['year','month'])['c2h6'].count()

### Get background 
Have to get the ch4 and bootstrapping 

In [ ]:
fh = 400
fw = 400

# NOAAoldandfresh_allyears

p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='', x_axis_type="log", y_axis_type="log")
colors = bokeh.palettes.d3['Category20'][20]


source = ColumnDataSource(dict(x=NOAAoldandfresh_allyears.c2h6.values*1e-3, 
                               y=NOAAoldandfresh_allyears.c3h8.values*1e-3, 
                               z=newdirct_all,
                               q=newspd_all/100))
mapper = linear_cmap(field_name='z', palette=Viridis[5],
                     low=min(newdirct_all) ,high=max(newdirct_all))
p.circle(x='x',y='y',line_color=mapper,color=mapper,fill_alpha=.5,source=source, size=10)
color_bar = ColorBar(color_mapper=mapper['transform'],
                     width=8, title='Wind direction (degrees)',
                     major_label_text_font_size="15pt",
                     title_standoff=20,
                     label_standoff=5,
                     title_text_font_size="15pt",
                     location=(0,0))
p.add_layout(color_bar, 'right')
p.xaxis.axis_label = "C₂H₆ (ppb)"
p.yaxis.axis_label = "C₃H₈ (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'

p.legend.location = "bottom_right"
bokeh.io.show(p)



In [ ]:
print(len(NOAAoldandfresh_allyears.loc[NOAAoldandfresh_allyears['c3h8'] < 1000]))
print(len(NOAAoldandfresh_allyears.loc[NOAAoldandfresh_allyears['c3h8'] > 1000]))

Paul: The background is not going to have a huge effect. The fresh emissions will inform most of the posterior anyway. So you should just choose a couple of values below the knee and can try using one or the other to get the background. So Don't worry about choosing a specific wind direction when choosing the background value. Just get an average point and then get the co-emitted ch4 corresponding to the index. 
 

In [ ]:
print(np.median(NOAAoldandfresh_allyears.loc[NOAAoldandfresh_allyears['c3h8'] < 1000]['c3h8']))
aged = NOAAoldandfresh_allyears.loc[NOAAoldandfresh_allyears['c3h8'] < 1000]

In [ ]:
vals = [x for x in abs(aged['c3h8'] - np.median(aged['c3h8']))]


In [ ]:
np.argmin(vals)

In [ ]:
vals[81]

In [ ]:
c3background_val = np.median(aged['c3h8'])
c3background_ind = np.argmin(vals)

In [ ]:
ch4background_val = aged.ch4.values[c3background_ind]
c2background_val = aged.c2h6.values[c3background_ind]

The lat/lon/time associated with this background number: 

In [ ]:
ind_bigarr = np.argmin(NOAAoldandfresh_allyears.c3h8 - c3background_val)

In [ ]:
backgroundinfo_df = NOAAoldandfresh_allyears.iloc[ind_bigarr]
backgroundinfo_df

Add an anomaly variable to the fresh emissions dataframe  

In [ ]:
NOAA_fresh_summerallyears['ch4_anomaly'] = NOAA_fresh_summerallyears.ch4.values - ch4background_val
NOAA_fresh_summerallyears['c3h8_anomaly'] = NOAA_fresh_summerallyears.c3h8.values - c3background_val
NOAA_fresh_summerallyears['c2h6_anomaly'] = NOAA_fresh_summerallyears.c2h6.values - c2background_val

In [ ]:
@numba.njit
def draw_bs_sample(data):
    """Draw a bootstrap sample from a 1D data set."""
    return np.random.choice(data, size=len(data))


@numba.njit
def draw_bs_pairs(x, y):
    """Draw a pairs bootstrap sample."""
    inds = np.arange(len(x))
    bs_inds = draw_bs_sample(inds)

    return x[bs_inds], y[bs_inds]

In [ ]:
def draw_bs_pairs_reps_slope(x, y, size=1):
    """
    Draw bootstrap pairs replicates.
    """
    slopereps = np.empty(size)
    interceptreps = np.empty(size)
    for i in range(size):
        slope, intercept, r_value, p_value, std_err = st.linregress(*draw_bs_pairs(x, y))
        slopereps[i] = slope
        
    return slopereps

In [ ]:
def draw_bs_threes(x, y, z):
    inds = np.arange(len(x))
    bs_inds = draw_bs_sample(inds)
    
    return x[bs_inds], y[bs_inds], z[bs_inds]

In [ ]:
def draw_bs_threes_reps(x, y, z, size=1):
    stdreps_x = np.empty(size)
    stdreps_y = np.empty(size)
    stdreps_z = np.empty(size)
    
    for i in range(size):
        xs, ys, zs = draw_bs_threes(x,y,z)
        stdreps_x[i] = np.std(xs)
        stdreps_y[i] = np.std(ys)
        stdreps_z[i] = np.std(zs)
    
    return stdreps_x, stdreps_y, stdreps_z

Get the CI for the correlation for use in the emissions file.

In [ ]:
# Get reps
bs_reps_C3C1 = draw_bs_pairs_reps_slope(NOAA_fresh_summerallyears['ch4_anomaly'].values,
                                        NOAA_fresh_summerallyears['c3h8_anomaly'].values*1e-3, size=10000)
bs_reps_C2C1 = draw_bs_pairs_reps_slope(NOAA_fresh_summerallyears['ch4_anomaly'].values,
                                        NOAA_fresh_summerallyears['c2h6_anomaly'].values*1e-3, size=10000)
print((np.percentile(bs_reps_C3C1, [2.5, 97.5])),'c3/c1 ppb/ppb pctile')
print((np.percentile(bs_reps_C2C1, [2.5, 97.5])),'c2/c1 ppb/ppb pctile')

Instead of this, I am going to use the 2017 average C3/C1 and C/C1 from the oklahoma site from my 2022 paper, since those included a lot more data (not just a few summer points) and the background was constructed using all points below 1000 ppt instead of choosing just a single one. SO NOT using the CI above for the emissions.

HOWEVER, do need to get the geophysical error for the points that you are using in the inversion. 

In [ ]:
bs_repsC3, bs_repsC2, bs_repsC1 = draw_bs_threes_reps(NOAA_fresh_summerallyears['c3h8_anomaly'].values*1e-3,
                                                     NOAA_fresh_summerallyears['c2h6_anomaly'].values*1e-3,
                                                     NOAA_fresh_summerallyears['ch4_anomaly'].values, size=10000)

In [ ]:
print('anomaly average standard deviation from pairs bootstrapping, units of ppb')
print(np.mean(bs_repsC3))
print(np.mean(bs_repsC2))
print(np.mean(bs_repsC1))

### Save out data needed for stilt 

In [ ]:
NOAA_fresh_summerallyears.columns

In [ ]:
noaaforstilt = NOAA_fresh_summerallyears[['lat','lon','dt_time','intake_height']]

In [ ]:
noaaforstilt.to_csv('/Users/arianatribby/git/oklahoma_propane/data/windrose/noaa_summer2017to2020summer_forstilt.csv')

Saving out the anomaly values in units of ppb 

In [ ]:
noaa_anomalystilt = NOAA_fresh_summerallyears[['lat','lon','dt_time','intake_height','c3h8_anomaly','c2h6_anomaly','ch4_anomaly']]
noaa_anomalystilt['c3h8_anomaly'] = NOAA_fresh_summerallyears['c3h8_anomaly']*1e-3
noaa_anomalystilt['c2h6_anomaly'] = NOAA_fresh_summerallyears['c2h6_anomaly']*1e-3


In [ ]:
noaa_anomalystilt.to_csv('/Users/arianatribby/git/oklahoma_propane/data/windrose/noaa_summer2017to2020summer_forstilt_withflasks.csv')

### Compare this windspeed and direction with meterology data used for STILT
Compare 3 days before (and all in between too) 
Ask if you can use Josh code to grab the HERR and ERA5 and GFS to compare them. Can use this to decide the met fields, and also to get the uncertainty in wind which will help inform the inversion uncertainty. 